In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
import statsmodels.api as sm
from sklearn import tree
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
# Load data
bank_df = pd.read_csv(r"E:\Michaelmas\BU1745-Foundational BA\Group Project\bank-full.csv")

In [ ]:
# Check for null values in each column
null_values = bank_df.isnull().sum()
print(null_values)

In [ ]:
bank_df.describe()

In [ ]:
bank_df.dtypes

In [ ]:
bank_df['y']=bank_df['y'].map({'yes': 1, 'no': 0}).astype(int)

#### Ques 1: Key customer characteristics and campaign factors impacting the likelihood of term deposit subscription

#### Exploratory Data Analysis

In [ ]:
# Percentage of Subscriptions by Month
# Order of months
month_order = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

# Convert 'month' column to categorical type
bank_df['month'] = pd.Categorical(bank_df['month'], categories=month_order, ordered=True)

# Percentage of 'yes' and 'no' responses for each month
subscription_counts = bank_df.groupby(['month', 'y']).size().unstack(fill_value=0)
subscription_percentages = subscription_counts.div(subscription_counts.sum(axis=1), axis=0) * 100

# Plot
subscription_percentages.plot(kind='bar', stacked=False, color = ['#1f77b4', '#aec7e8'], figsize=(8, 7))
# Adding labels and title
plt.title('Percentage of Subscriptions ("yes" and "no") by Month')
plt.xlabel('Month')
plt.ylabel('Percentage of Subscriptions')
plt.legend(title='Subscription Status', labels=['No', 'Yes'])
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Binning the 'age' column 
bank_df['age_group'] = pd.cut(bank_df['age'], bins=[0, 25, 40, 60, 100], labels=['young', 'middle-aged', 'older', 'senior'])

# Binning the 'balance' column
bank_df['balance_group'] = pd.cut(bank_df['balance'], bins=[0, 1000, 10000, 100000], labels=['low', 'medium', 'high'])

bank_df[['age', 'age_group', 'balance', 'balance_group']].head()

In [ ]:
# Visualizing the relationship between 'age_group' and subscription (y)
plt.figure(figsize=(6, 4))
sns.countplot(data=bank_df, x='age_group', hue='y', palette='Blues')
plt.title('Term Deposit Subscription by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.legend(title='Subscription', labels=['No Subscription (y=0)', 'Subscription (y=1)'])
plt.show()

# Visualizing the relationship between 'balance_group' and subscription (y)
plt.figure(figsize=(6, 4))
sns.countplot(data=bank_df, x='balance_group', hue='y', palette='Blues')
plt.title('Term Deposit Subscription by Balance Group')
plt.xlabel('Balance Group')
plt.ylabel('Count')
plt.legend(title='Subscription', labels=['No Subscription (y=0)', 'Subscription (y=1)'])
plt.show()

In [ ]:
# Distribution of the 'balance' variable - to check if we need transformation while modeling
plt.figure(figsize=(6, 4))
sns.histplot(bank_df['balance'], bins=30, kde=True, color='skyblue')
plt.xlabel("Balance")
plt.ylabel("Frequency")
plt.title("Distribution of Balance")
plt.show()

In [ ]:
# Distribution of the 'duration' variable - to check if we need transformation while modeling (Data is rightly skewed)
plt.figure(figsize=(6, 4))
sns.histplot(bank_df['duration'], bins=30, kde=True, color='skyblue')
plt.xlabel("Call Duration (seconds)")
plt.ylabel("Frequency")
plt.title("Distribution of Call Duration")
plt.show()

In [ ]:
subscribed_durations = bank_df[bank_df['y'] == 1]['duration']
non_subscribed_durations = bank_df[bank_df['y'] == 0]['duration']

# Calculate the count of subscriptions and non-subscriptions for each duration
duration_counts_subscribed = subscribed_durations.value_counts().sort_index()
duration_counts_non_subscribed = non_subscribed_durations.value_counts().sort_index()

# Plotting both to see the distributions side-by-side
plt.figure(figsize=(12, 6))
plt.plot(duration_counts_subscribed.index, duration_counts_subscribed.values, label='Subscribed (y=1)', color='blue')
plt.plot(duration_counts_non_subscribed.index, duration_counts_non_subscribed.values, label='Not Subscribed (y=0)', color='skyblue')
plt.xlabel("Call Duration (seconds)")
plt.ylabel("Frequency")
plt.title("Distribution of Call Duration for Subscribed vs. Not Subscribed")
plt.legend()
plt.show()

# Can remove records where duration is more than 2000. Data loss is minimal

In [ ]:
bank_df['job'].value_counts().sort_index()
# Unknown jobs accounts for only 0.64% of data

In [ ]:
# Term deposit subscription by job type
bank_df['y_subscribed'] = bank_df['y'].map({1: 'Subscribed', 0: 'Not Subscribed'})
job_subscription_counts = bank_df.groupby(['job', 'y_subscribed']).size().unstack(fill_value=0)

# Plot
plt.figure(figsize=(14, 8))
job_subscription_counts.plot(kind='bar', stacked=True, color=['#1f77b4', '#ff7f0e'], width=0.8)
plt.xlabel("Job Type")
plt.ylabel("Number of People")
plt.title("Term Deposit Subscription Status by Job Type")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Subscription Status")
plt.show()

In [ ]:
# Drop the unnnecessary columns
bank_df = bank_df.drop(['age_group', 'balance_group', 'y_subscribed'], axis=1)

In [ ]:
# Occurrences of campaign
campaign_counts = bank_df['campaign'].value_counts().sort_index()

# Rate of term deposit subscriptions
subscription_rate = bank_df['y'].value_counts(normalize=True) * 100

print("Number of campaigns contact counts for each campaign:-")
print(campaign_counts)

In [ ]:
# Plotting the distribution of 'campaign' by subscription status
sns.histplot(data=bank_df, x='campaign', hue='y', bins=20, kde=True, palette='YlGnBu', edgecolor='black')
plt.title('Distribution of Campaign Contacts by Subscription Status')
plt.xlabel('Number of Contacts (campaign)')
plt.ylabel('Frequency')
plt.legend(title='Subscription Status', labels=['No', 'Yes'])

plt.show()

In [ ]:
# Campaign values with counts >= 100
valid_campaigns = campaign_counts[campaign_counts >= 100].index

# Filter the DataFrame to include only rows where the campaign is in the valid_campaigns
# and where the jobtype is  'unknown' 
filtered_bank_df = bank_df[(bank_df['campaign'].isin(valid_campaigns)) & (bank_df['job'] != 'unknown') & (bank_df['duration'] <= 2000)]

In [ ]:
filtered_bank_df.shape

In [ ]:
data = filtered_bank_df.copy()
data['subscription'] = data['y']

# Bins for balance and duration
balance_bins = [0, 2500, 3000, 5000, 10000] 
duration_bins = [0, 300, 400, 600, 900, 1200] 

balance_labels = ['<2500', '2500-3000', '3000-5000', '>5000']
duration_labels = ['<300', '300-400', '400-600', '600-900', '>900']
data['balance_bin'] = pd.cut(data['balance'], bins=balance_bins, labels=balance_labels)
data['duration_bin'] = pd.cut(data['duration'], bins=duration_bins, labels=duration_labels)

# Subscription rate
subscription_rate = data.groupby(['balance_bin', 'duration_bin'])['subscription'].mean().unstack()

# Plot
plt.figure(figsize=(10, 6))
sns.heatmap(subscription_rate, annot=True, fmt=".2%", cmap="YlGnBu", cbar_kws={'label': 'Subscription Rate (%)'})
plt.xlabel("Call Duration (seconds)")
plt.ylabel("Balance (€)")
plt.title("Subscription Rate by Balance and Call Duration")
plt.show() 

In [ ]:
# Dummies for categorical variables
bank_dummies_df = pd.get_dummies(filtered_bank_df, drop_first = True, dtype = int)

In [ ]:
# Correlation matrix of the variables

In [ ]:
corr_matrix = bank_dummies_df.drop(['y'], axis=1).corr()

plt.figure(figsize=(26, 20))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Matrix of Variables")
plt.show()

# Highly correlated feature pairs
threshold = 0.8
high_corr_pairs = []
for col1 in corr_matrix.columns:
    for col2 in corr_matrix.columns:
        if col1 != col2 and abs(corr_matrix.loc[col1, col2]) > threshold:
            pair = tuple(sorted([col1, col2]))  # Sort to avoid duplicate pairs
            if pair not in high_corr_pairs:
                high_corr_pairs.append(pair + (corr_matrix.loc[col1, col2],))

print("Highly Correlated Feature Pairs (Correlation > 0.8):")
for pair in high_corr_pairs:
    print(f"{pair[0]} - {pair[1]}: Correlation = {pair[2]:.2f}")

In [ ]:
# Checking for multi-colinearity via Variance Inflation Factor

In [ ]:
X = bank_dummies_df.drop(columns='y') 

# VIF for each feature
vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

In [ ]:
# Feature importance

In [ ]:
# Segregating independent and target variables
x = bank_dummies_df.drop(columns='y')  
y = bank_dummies_df['y'] 

# Fit the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=19)
clf.fit(x, y)

feature_importances = pd.DataFrame({
    'feature': x.columns,
    'importance': clf.feature_importances_
}).sort_values(by='importance', ascending=False)

print(feature_importances)

In [ ]:
# Visualizing the feature importances
plt.figure(figsize=(14, 11))
plt.barh(feature_importances['feature'], feature_importances['importance'], color='royalblue')
plt.xlabel('Importance')
plt.title('Feature Importance from Decision Tree Classifier')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
# Selecting relevant columns for modeling

In [ ]:
# Removing poutcome_unknown as its VIF is high and is also highly co-related to pdays. Also, pdays has more feature importance compared to poutcome_unknown
# Removing education, marital, loan_yes, default_yes and their encoded variables as their feature importance is less than 0.01
bank_final_df = bank_dummies_df[
                                 [
                                      'age', 'balance', 'duration', 'campaign', 'pdays', 'day','previous',
                                      'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
                                      'job_management', 'job_retired', 'job_self-employed', 'job_services',
                                      'job_student', 'job_technician', 'job_unemployed', 'housing_yes', 
                                      'contact_telephone', 'contact_unknown', 'month_feb',
                                      'month_mar', 'month_apr', 'month_may', 'month_jun', 'month_jul',
                                      'month_aug', 'month_sep', 'month_oct', 'month_nov', 'month_dec',
                                      'poutcome_other', 'poutcome_success', 'y'
                                 ]
                               ]

In [ ]:
bank_final_df['y'].value_counts().sort_index()

In [ ]:
# Balancing the data by duplicating records as number of 1's(11.84%) are very less compared to 0's(88.16%)

In [ ]:
# Separate the data into majority and minority classes
data_majority = bank_final_df[bank_final_df['y'] == 0]  # Adjust if 0 is the majority class
data_minority = bank_final_df[bank_final_df['y'] == 1]  # Adjust if 1 is the minority class

# Calculating duplication factor
duplication_factor = len(data_majority) // len(data_minority)

# Duplicate the minority class
data_minority_duplicated = pd.concat([data_minority] * duplication_factor, ignore_index=True)

# Combining the majority class and duplicated minority class
balanced_df = pd.concat([data_majority, data_minority_duplicated], ignore_index=True)

balanced_data_majority = balanced_df[balanced_df['y'] == 0]
balanced_data_minority = balanced_df[balanced_df['y'] == 1]

print("Number of majority class instances after balancing:", len(balanced_data_majority))
print("Number of minority class instances after balancing:", len(balanced_data_minority))
print("Balanced dataset shape:", balanced_df.shape)

In [ ]:
# Target and predictors variable(s)
X = balanced_df[
                    [
                          'age', 'balance', 'duration', 'campaign', 'pdays', 'day','previous',
                          'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
                          'job_management', 'job_retired', 'job_self-employed', 'job_services',
                          'job_student', 'job_technician', 'job_unemployed', 
                          'housing_yes', 'contact_telephone', 'contact_unknown', 'month_feb',
                          'month_mar', 'month_apr', 'month_may', 'month_jun', 'month_jul',
                          'month_aug', 'month_sep', 'month_oct', 'month_nov', 'month_dec',
                          'poutcome_other', 'poutcome_success'
                    ]
               ]

y = balanced_df['y']

In [ ]:
# Logistic Regression model

In [ ]:
# Define and scale numerical features for Logistic Regression
numerical_features = ['age', 'balance', 'duration', 'campaign', 'pdays', 'day', 'previous']
scaler = StandardScaler()

# Duration data is rightly skewed
X['duration'] = np.log1p(X['duration'])  # log1p handles zero values by computing log(1 + x)
X_scaled = X.copy()
X_scaled[numerical_features] = scaler.fit_transform(X_scaled[numerical_features])

# Train-Test Split - 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=19, stratify=y)

# Initialize and train Logistic Regression model
logreg = LogisticRegression(max_iter=1000, random_state=19, class_weight='balanced')
logreg.fit(X_train, y_train)

# Predictions
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_report)
print("\nROC- Area under curve:\n", roc_auc)

# Confusion Matrix Plot
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# ROC Curve Plot
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()

# Coefficients
coefficients = logreg.coef_[0]
feature_names = X_scaled.columns

# Odds and Probability calculation
odds = np.exp(coefficients)  # Odds ratio
probability = odds / (1 + odds)  # Probability from odds

# DataFrame of features, coefficients, odds, and probability
feature_summary = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Odds': odds,
    'Probability': probability
})

# Logit Equation
intercept = logreg.intercept_[0]
print(f"\nLogit Equation: log(odds) = {intercept:.4f} + " + " + ".join([f"{coef:.4f}*{name}" for coef, name in zip(coefficients, feature_names)]))

# Feature Summary
print("\nFeature Summary with Coefficient, Odds, and Probability:")
print(feature_summary)

In [ ]:
# Summary of logistic Regression

In [ ]:
# Add an intercept column to X_train for statsmodels (sklearn adds it automatically, but statsmodels does not)
X_train_sm = sm.add_constant(X_train)  # Adds a constant term to the predictors

# Fit logistic regression model using statsmodels
logit_model = sm.Logit(y_train, X_train_sm)
result = logit_model.fit()

print(result.summary())

# Extracting coefficients, p-values, and other statistics into a DataFrame
summary_df = pd.DataFrame({
    "Feature": X_train_sm.columns,
    "Coefficient": result.params.values,
    "P-value": result.pvalues.values
})

print(summary_df)

In [ ]:
# Filter significant features based on their p-value
significant_features = summary_df[summary_df['P-value'] < 0.05].sort_values(by="Coefficient", key=abs, ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(significant_features['Feature'], significant_features['Coefficient'], color=np.where(significant_features['Coefficient'] > 0, 'royalblue', 'skyblue'))
plt.xlabel("Coefficient Value")
plt.title("Significant Features Affecting Term Deposit Subscription Likelihood (p < 0.05)")
plt.gca().invert_yaxis()  
plt.show()

In [ ]:
# Decision Tree Classifier

In [ ]:
# Define the depth of the tree for model tuning and performance
max_depth = 10  

# Initialize and train the Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=19, max_depth=max_depth, class_weight='balanced')
dt_clf.fit(X_train, y_train)

# Predictions
y_pred = dt_clf.predict(X_test)
y_pred_proba = dt_clf.predict_proba(X_test)[:, 1]  

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Results
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_report)
print("\nROC- Area under curve:\n", roc_auc)

# Confusion Matrix Plot
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# ROC Curve Plot
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--') 
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()

# Save predictions to CSV because the accuracy of Decision Tree is more 
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Predicted_Probability': y_pred_proba
})

predictions_df.to_csv('term_deposit_predictions.csv', index=False)

In [ ]:
importances = dt_clf.feature_importances_

# DataFrame of features and their scores
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df.head(10))

In [ ]:
# Plot Partial Dependence for the top features
PartialDependenceDisplay.from_estimator(dt_clf, X_train, features=[2, 32, 19])  
plt.suptitle("Partial Dependence of Key Features on Subscription Likelihood")
plt.show()

In [ ]:
# ROC for Logistic Regression
fpr_logreg, tpr_logreg, _ = roc_curve(y_test, logreg.predict_proba(X_test)[:, 1])
roc_auc_logreg = roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1])

# ROC for Decision Tree
fpr_dt, tpr_dt, _ = roc_curve(y_test, dt_clf.predict_proba(X_test)[:, 1])
roc_auc_dt = roc_auc_score(y_test, dt_clf.predict_proba(X_test)[:, 1])

# Plotting both ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_logreg, tpr_logreg, label=f"Logistic Regression (AUC = {roc_auc_logreg:.2f})", color="blue")
plt.plot(fpr_dt, tpr_dt, label=f"Decision Tree (AUC = {roc_auc_dt:.2f})", color="green")
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison: Logistic Regression vs Decision Tree")
plt.legend(loc="lower right")
plt.show()

#### Ques 2: Seasonal trends in call duration, monthly patterns, and subscription counts for predicting the likelihood of a customer subscribing to a term deposit

In [ ]:
bank_q2_df = bank_df.copy()

In [ ]:
monthly_data = bank_q2_df.groupby('month').agg(
                                                avg_duration=('duration', 'mean'),
                                                subscriber_count=('y', 'sum')
                                              ).reset_index()
monthly_data

In [ ]:
# Data Analysis & Visualization

In [ ]:
# Ordering month as a categorical variable
bank_q2_df['month'] = pd.Categorical(
                                        bank_q2_df['month'],
                                        categories=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                                        ordered=True
                                    )

# Calculating monthly average duration and subscriber counts
monthly_data = bank_q2_df.groupby('month').agg(
                                                avg_duration=('duration', 'mean'),
                                                subscriber_count=('y', 'sum')
                                              ).reset_index()

# Plot
plt.figure(figsize=(10, 5))

# Monthly average duration (line plot for trend)
sns.lineplot(data=monthly_data, x='month', y='avg_duration', marker='o', color='blue', label="Average Call Duration(seconds)")

# Monthly subscriber count (bar plot)
sns.barplot(data=monthly_data, x='month', y='subscriber_count', color='royalblue', alpha=0.8, label="Subscriber Count")

# Background shading for seasonal periods
plt.axvspan(4, 7, color='peachpuff', alpha=0.3, label="Summer Months (May-Aug)")
plt.axvspan(9, 11, color='lightblue', alpha=0.3, label="Holiday Season (Oct-Dec)")
plt.xlabel("Month")
plt.ylabel("Values")
plt.title("Seasonal Trends: Average Call Duration and Subscriber Count by Month")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Creating dummies for categorical variables
bank_q2_dummies_df = pd.get_dummies(bank_q2_df, columns=['month'], drop_first=True)

In [ ]:
# Correlation analysis of features

In [ ]:
# Seasonal indicators for summer and holiday seasons
bank_q2_dummies_df['is_summer'] = bank_q2_dummies_df[['month_may', 'month_jun', 'month_jul', 'month_aug']].sum(axis=1).astype(bool).astype(int)
bank_q2_dummies_df['is_holiday_season'] = bank_q2_dummies_df[['month_oct', 'month_nov', 'month_dec']].sum(axis=1).astype(bool).astype(int)

# Define features and target
features = ['duration', 'is_summer', 'is_holiday_season'] + [col for col in bank_q2_dummies_df.columns if col.startswith('month_')]
X = bank_q2_dummies_df[features]
y = bank_q2_dummies_df['y'] 

corr_matrix = X.corr()

# Correlation matrix plot
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Matrix of Independent Variables")
plt.show()

# Identifying highly correlated pairs
threshold = 0.8
high_corr_pairs = [(col1, col2, corr_matrix.loc[col1, col2]) 
                   for col1 in corr_matrix.columns 
                   for col2 in corr_matrix.columns 
                   if col1 != col2 and abs(corr_matrix.loc[col1, col2]) > threshold]

print("Highly Correlated Feature Pairs (Correlation > 0.8):")
for pair in high_corr_pairs:
    print(f"{pair[0]} - {pair[1]}: Correlation = {pair[2]:.2f}")

In [ ]:
bank_q2_final_df = bank_q2_dummies_df[
                                        [
                                           'duration', 'is_summer', 'is_holiday_season', 'month_feb', 'month_mar',
                                           'month_apr', 'month_may', 'month_jun', 'month_jul', 'month_aug',
                                           'month_sep', 'month_oct', 'month_nov', 'month_dec', 'y'
                                        ]
                                    ]

In [ ]:
bank_q2_final_df['y'].value_counts().sort_index()

In [ ]:
# Balancing the data

In [ ]:
# Separate the data into majority and minority classes
data_majority = bank_q2_final_df[bank_q2_final_df['y'] == 0]  # Adjust if 0 is the majority class
data_minority = bank_q2_final_df[bank_q2_final_df['y'] == 1]  # Adjust if 1 is the minority class

# Calculating duplication factor
duplication_factor = len(data_majority) // len(data_minority)

# Duplicate the minority class
data_minority_duplicated = pd.concat([data_minority] * duplication_factor, ignore_index=True)

# Combining the majority class and duplicated minority class
balanced_q2_df = pd.concat([data_majority, data_minority_duplicated], ignore_index=True)

balanced_data_majority = balanced_q2_df[balanced_q2_df['y'] == 0]
balanced_data_minority = balanced_q2_df[balanced_q2_df['y'] == 1]

print("Number of majority class instances after balancing:", len(balanced_data_majority))
print("Number of minority class instances after balancing:", len(balanced_data_minority))
print("Balanced dataset shape:", balanced_q2_df.shape)

In [ ]:
# Defining the independent and dependent variables for modeling
X = balanced_q2_df[features]
y = balanced_q2_df['y']

In [ ]:
# Log transform the 'duration' feature
X['duration'] = np.log1p(X['duration'])  # log1p applies log(1 + x) to handle zero values

# Scaling numerical features
scaler = StandardScaler()
X[['duration']] = scaler.fit_transform(X[['duration']])  # Scaling only 'log_duration'

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19, stratify=y)

# Initialize and train Logistic Regression model
logreg = LogisticRegression(max_iter=1000, random_state=19, class_weight='balanced')
logreg.fit(X_train, y_train)

# Predictions
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("Accuracy:", accuracy)
print("ROC-AUC Score:", roc_auc)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# ROC-AUC metrics
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="green", label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--') 
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

# Coefficients from the logistic regression model
intercept = logreg.intercept_[0]  
coefficients = logreg.coef_[0]    
feature_names = X.columns 

# DataFrame to display the coefficients, odds, and probabilities
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Odds': np.exp(coefficients),  # Odds = exp(Coefficient)
    'Probability': np.exp(coefficients) / (1 + np.exp(coefficients))  # Probability from odds
})

# Coefficients
coef_df = coef_df.reindex(coef_df['Coefficient'].abs().sort_values(ascending=False).index)
print("\nSorted Coefficients (by absolute value):")
print(coef_df)

# logit equation
logit_equation = f"log(odds) = {intercept:.4f} + " + " + ".join([f"{coef:.4f}*{name}" for coef, name in zip(coefficients, feature_names)])
print("\nLogit Equation:")
print(logit_equation)

# DataFrame to save actual values, predicted values, and predicted probabilities
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Predicted_Probability': y_pred_proba
})

# Save to CSV file
predictions_df.to_csv('term_deposit_predictions_by_seasonality.csv', index=False)

In [ ]:
# Segmenting customers based on likelihood of subscribing

In [ ]:
X['predicted_probability'] = logreg.predict_proba(X)[:, 1] 

# Segments based on probability thresholds
X['subscription_likelihood'] = pd.cut(X['predicted_probability'], bins=[0, 0.5, 0.75, 1], 
                                      labels=['Less Likely', 'Moderately Likely', 'Highly Likely'])

print(X['subscription_likelihood'].value_counts())

In [ ]:
# Season column based on the 'is_summer' and 'is_holiday_season' 
X['season'] = np.select(
    [
        X['is_summer'] == 1,
        X['is_holiday_season'] == 1
    ],
    [
        'Summer',
        'Holiday Season'
    ]
)

# Aggregate data to count the number of customers in each likelihood segment per season
seasonal_likelihood_counts = X.groupby(['season', 'subscription_likelihood']).size().reset_index(name='customer_count')

seasonal_order = ['Summer', 'Holiday Season']
seasonal_likelihood_counts['season'] = pd.Categorical(seasonal_likelihood_counts['season'], categories=seasonal_order, ordered=True)

# Plot the data
plt.figure(figsize=(7, 4))
sns.barplot(data=seasonal_likelihood_counts, x='season', y='customer_count', hue='subscription_likelihood', palette="Blues", width = 0.3)
plt.title("Customer Subscription Likelihood Segments by Season")
plt.xlabel("Season")
plt.ylabel("Customer Count")
plt.legend(title="Likelihood Segment", loc='upper right')
plt.show()


#### Ques 3 Financial and demographic characteristics of clients who are likely to have a housing loan

In [ ]:
bank_q3_df = bank_df.copy()

In [ ]:
bank_q3_df = bank_q3_df[['balance', 'loan', 'default', 'y', 'marital', 'age', 'housing']]

In [ ]:
# Distribution of `housing`
plt.figure(figsize=(5, 3))
sns.countplot(data=bank_df, x='housing', width=0.3)
plt.title("Distribution of Housing Loan")
plt.xlabel("Housing Loan")
plt.ylabel("Count")
plt.show()

In [ ]:
# Housing loan by age
plt.figure(figsize=(9, 3))
sns.histplot(data=bank_q3_df, x='age', hue='housing', multiple='stack', bins=20)
plt.title("Distribution of Housing Loan by Age")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.figure(figsize=(5, 2))
sns.countplot(data=bank_q3_df, x='marital', hue='housing',  width =0.3)
plt.title("Housing Loan by Marital Status")
plt.xlabel("Marital Status")
plt.ylabel("Count")
plt.show()

In [ ]:
# Distribution of 'housing' loan status by 'loan' status
loan_housing_counts = bank_q3_df.groupby(['housing', 'loan']).size().unstack(fill_value=0)

# Convert counts to percentages
loan_housing_percentages = loan_housing_counts.div(loan_housing_counts.sum(axis=1), axis=0) * 100

# Plot
fig, ax = plt.subplots(figsize=(7, 4))
loan_housing_percentages.plot(kind='bar', stacked=True, width=0.3, ax=ax)  

plt.title("Percentage Distribution of Personal Loan Status with Housing Loan")
plt.xlabel("Housing Loan")
plt.ylabel("Percentage")
plt.legend(title="Personal Loan Status", labels=['No Personal Loan', 'Personal Loan'])
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Converting housing to binary
bank_q3_df['housing'] = bank_q3_df['housing'].map({'yes': 1, 'no': 0}).astype(int)

In [ ]:
if 'y' in bank_q3_df.columns:
    bank_q3_df = bank_q3_df.rename(columns={'y': 'term_deposit'})

In [ ]:
bank_q3_dummies = pd.get_dummies(bank_q3_df, drop_first=True)

In [ ]:
bank_q3_dummies.columns

In [ ]:
# Correlation Matrix of features

In [ ]:
correlation_matrix = bank_q3_dummies.drop(columns = 'housing').corr()

# Plot the Correlation Matrix
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix for Features")
plt.show()

# Identifying highly correlated pairs
threshold = 0.8
high_corr_pairs = [(col1, col2, correlation_matrix.loc[col1, col2]) 
                   for col1 in correlation_matrix.columns 
                   for col2 in correlation_matrix.columns 
                   if col1 != col2 and abs(correlation_matrix.loc[col1, col2]) > threshold]

print("Highly Correlated Feature Pairs (Correlation > 0.8):")
for pair in high_corr_pairs:
    print(f"{pair[0]} - {pair[1]}: Correlation = {pair[2]:.2f}")

In [ ]:
# Define dependent variable and predictors
X = bank_q3_dummies.drop(columns='housing')
y = bank_q3_dummies['housing']

#  Fit the Decision Tree Classifier 
tree_model = DecisionTreeClassifier(random_state=19)
tree_model.fit(X, y)

# Extract and Plot Feature Importance
feature_importances = pd.Series(tree_model.feature_importances_, index=X.columns)
feature_importances = feature_importances.sort_values(ascending=False)
feature_importance_df = feature_importances.reset_index()
feature_importance_df.columns = ['Feature', 'Importance']
print("Feature Importances for Predicting Housing Loan:")
print(feature_importance_df)

# Plot the feature importances
plt.figure(figsize=(8, 4))
sns.barplot(x=feature_importances, y=feature_importances.index, palette="Blues")
plt.title("Feature Importance for Predicting Housing Loan Ownership")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
bank_q3_df['housing'].value_counts().sort_index() # More or less balanced data - No need to do balancing

In [ ]:
# Logistic Regression modeling

In [ ]:
# Define the features and target
X = bank_q3_dummies.drop(columns='housing')
y = bank_q3_dummies['housing']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19, stratify=y)

# Scale the numerical variables
scaler = StandardScaler()
X_train[['balance', 'age']] = scaler.fit_transform(X_train[['balance', 'age']])
X_test[['balance', 'age']] = scaler.transform(X_test[['balance', 'age']])

# Initialize and train the logistic regression model
logreg = LogisticRegression(max_iter=1000, random_state=19, class_weight='balanced')
logreg.fit(X_train, y_train)

# Predictions
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]  # Probabilities for ROC-AUC calculation

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("Accuracy:", accuracy)
print("ROC-AUC Score:", roc_auc)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Coefficients from the logistic regression model
intercept = logreg.intercept_[0]
coefficients = logreg.coef_[0]
feature_names = X.columns

# DataFrame with feature details, including odds and probability
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Odds': np.exp(coefficients),  # Odds = exp(Coefficient)
    'Probability': np.exp(coefficients) / (1 + np.exp(coefficients))  # Probability from odds
})

# Coefficients
coef_df = coef_df.reindex(coef_df.Coefficient.abs().sort_values(ascending=False).index)
print("\nSorted Coefficients:")
print(coef_df)

# Logit equation
logit_equation = f"log(odds) = {intercept:.4f} " + " + ".join([f"{coef:.4f}*{name}" for coef, name in zip(coefficients, feature_names)])
print("\nLogit Equation:")
print(logit_equation)

In [ ]:
# Predictors and target
X = bank_q3_dummies.drop(columns='housing')
y = bank_q3_dummies['housing']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19, stratify=y)

# Scaling numerical variables
scaler = StandardScaler()
X_train[['balance', 'age']] = scaler.fit_transform(X_train[['balance', 'age']])
X_test[['balance', 'age']] = scaler.transform(X_test[['balance', 'age']])

# Initialize and train the Decision Tree model
dt_clf = DecisionTreeClassifier(max_depth=3, random_state=19, class_weight='balanced')
dt_clf.fit(X_train, y_train)

# Predictions
y_pred = dt_clf.predict(X_test)
y_pred_proba = dt_clf.predict_proba(X_test)[:, 1] 

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("Decision Tree Accuracy:", accuracy)
print("Decision Tree ROC-AUC Score:", roc_auc)
print("\nDecision Tree Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,3))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Decision Tree Confusion Matrix")
plt.show()

# Plot the Decision Tree
plt.figure(figsize=(20, 10))
plt.set_cmap("YlGnBu")
tree.plot_tree(dt_clf, feature_names=X.columns, class_names=['No Housing Loan', 'Yes Housing Loan'], filled=True)
plt.title("Decision Tree Visualization")
plt.show()

predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Predicted_Probability': y_pred_proba
})

# Save to a CSV file
predictions_df.to_csv('housing_loan_predictions.csv', index=False)

In [ ]:
# ROC for Logistic Regression
fpr_logreg, tpr_logreg, _ = roc_curve(y_test, logreg.predict_proba(X_test)[:, 1])
roc_auc_logreg = roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1])

# ROC for Decision Tree
fpr_dt, tpr_dt, _ = roc_curve(y_test, dt_clf.predict_proba(X_test)[:, 1])
roc_auc_dt = roc_auc_score(y_test, dt_clf.predict_proba(X_test)[:, 1])

# Plotting both ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_logreg, tpr_logreg, label=f"Logistic Regression (AUC = {roc_auc_logreg:.2f})", color="blue")
plt.plot(fpr_dt, tpr_dt, label=f"Decision Tree (AUC = {roc_auc_dt:.2f})", color="green")
plt.plot([0, 1], [0, 1], 'k--')  
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison: Logistic Regression vs Decision Tree")
plt.legend(loc="lower right")
plt.show()